# Assignment 1 for Genome Science (BBMS3009)
This assignment aims to understand the normalization of RNA-seq count data and the basics of the statistical tests behind Differential Gene expression detection.

* Student Name:
* Student ID:

**Requirement and Marks**

* This assignment covers 25 marks, including the completeness of whole script/notebook for 5 marks.
* You should submit a report in PDF (or MS Word) and a generated notebook (in .html) by Knit. Please answer the questions in the report with attaching the relevant figures, and keep your added script in the notebook.

**Expected running time on laptop**

* `DESeq2` installation: ~10min
* Running the whole notebook: ~5min
* Reading and thinking may take a few hours.

**Main reference**:

* [Chapter 8 in Modern Statistics for Modern Biology](https://web.stanford.edu/class/bios221/book/Chap-CountData.html)
* [Chapter 8 in Computational Genomics with R](http://compgenomr.github.io/book/rnaseqanalysis.html)

## Part 0. Environment setting
Installing the `DESeq2` package for loading data.

* Think: how the package `DESeq2` is installed? Through which platform and via which package?
* Think: what does the `if ... else` doing here?

In [ ]:
if (!requireNamespace("DESeq2", quietly = TRUE)) {
  if (!requireNamespace("BiocManager", quietly = TRUE)){
    install.packages("BiocManager")
  }
  BiocManager::install("DESeq2")
}

## Part 1. Load data

#### Load count matrix
gene-by-sample, with additional column for gene length. Namely the last column is the length of the gene.

In [ ]:
counts_file <- 'SRP029880.raw_counts.tsv'

counts <- as.matrix(read.csv(counts_file, header = T, sep = '\t'))

In [ ]:
dim(counts)
colnames(counts)

In [ ]:
summary(counts[,1:3])

#### Load colData for sample information

In [ ]:
col_data_file <- 'SRP029880.colData.tsv'

col_data <- read.table(col_data_file, header = T, sep = '\t', stringsAsFactors = TRUE)

In [ ]:
col_data

## 2. Normalization
For gene length bias correction, is often included in the "normalization" term.

### 2.1 CPM: Count per million
Normalization to the library size

In [ ]:
cpm <- t(t(counts[, 1:10]) / colSums(counts[, 1:10])) * 10^6

In [ ]:
head(cpm)

In [ ]:
hist(log(cpm[, 10] + 0.1), breaks=seq(-3, 11, 0.5))

### 2.2 RPKM: Read per killobase per million
Correction the gene length bias. It also often contained in the normalization term.

In [ ]:
geneLengths <- counts[, 11]
head(geneLengths)

<font color='red'>**Q1.1: Define RPKM in script, both in script below and in report [1 mark]**</font>

In [ ]:
# Q1.1 TODO: define rpkm
# rpkm <- PLEASE_FILL_HERE


In [ ]:
head(rpkm)

In [ ]:
hist(log(rpkm[, 10] + 0.1), breaks=seq(-3, 11, 0.5))

### 2.3 TPM: transcript per million
<font color='red'>**Q1.2: Define TPM in script, both in script below and in report [1 mark]**</font>

In [ ]:
# Q1.2 TODO: define tpm
# tpm <- PLEASE_FILL_HERE


In [ ]:
head(tpm)

### 2.4 Comparing RPKM and TPM
Within one sample comparison.

In [ ]:
plot(log10(rpkm[, 10]), log10(tpm[, 10]))

Think: ratio between TPM and RPKM across samples. Are they always the same for all samples? Why?

In [ ]:
# remove the zero values befor checking the ratio
idx = rowMeans(rpkm > 0) == 1
colMeans(tpm[idx, ] / rpkm[idx, ])

<font color='red'>**Q1.3: Describe the relationship and difference between RPKM and TPM, both within a sample and across multiple samples? Write in report [2 marks]**</font>

## 3. Differential gene expression

Now, you are going to detect differential expressed genes with `DEseq2` package. We aim to provide necessary scripts, so you don't need to read the detailed documentation of this package. But still feel free to do so.

* [DESeq2 package page](https://bioconductor.org/packages/release/bioc/html/DESeq2.html)
* [DESeq2 vignettes](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html)

In [ ]:
library(DESeq2)
library(stats)
library(ggplot2)

In [ ]:
#define the design formula
designFormula <- "~ group"

#create a DESeq dataset object from the count matrix and the colData 
dds <- DESeqDataSetFromMatrix(countData = counts[, 1:10], 
                              colData = col_data, 
                              design = as.formula(designFormula))
                              
#print dds object to see the contents
print(dds)

In [ ]:
size_factor <- DESeq2::estimateSizeFactorsForMatrix(dds@assays@data$counts)
total_reads <- colSums(dds@assays@data$counts)

<font color='red'>**Q2: Please plot the scatter plot between learned size factor and total reads. What is the relationship between the learned size factor and total reads? Please write it in report with attaching the figure [2 marks]**</font>

In [ ]:
# Q2 TODO: write script for scatter plot here, ideally with ggplot



Hints: see the [Fig. 8.1](https://web.stanford.edu/class/bios221/book/Chap-CountData.html#fig:rnaseq-normalization) in the book 
[Modern Statistics for Modern Biology](https://web.stanford.edu/class/bios221/book/Chap-CountData.html).

#### Remove lowly expressed genes
This step can reduce the number of tests, but be careful as some informative genes may indeed have low expression

Here, we only remove genes with no expression at all

In [ ]:
#For each gene, we count the total number of reads for that gene in all samples 
#and remove those that don't have at least 1 read. 

dds <- dds[ rowSums(dds@assays@data$counts) > 1, ]
dim(dds)

In [ ]:
sum(rowSums(dds@assays@data$counts) > 1)

### 3.1 Perform DE analysis

In [ ]:
dds <- DESeq(dds)

In [ ]:
#compute the contrast for the 'group' variable where 'CTRL' 
#samples are used as the control group. 

DEresults = results(dds, contrast = c("group", 'CASE', 'CTRL'))

#sort results by increasing p-value
DEresults <- as.data.frame(DEresults[order(DEresults$pvalue), ])

In [ ]:
#shows the top results
head(DEresults)

#### Visualize the DE results
plotting the MA plot

In [ ]:
DESeq2::plotMA(object = dds, ylim = c(-5, 5))

<font color='red'>**Q3: Plot the distribution p value below. If there is no genuine differentially expressed genes, what distribution of p values do you expected to see? Is there any range of the p value matching this expectation? Write in report with attaching the figure. [2 marks]**</font>

Hints: plotting the distribution of p values with sufficient number of bins, e.g., `geom_histogram(bins = 100)`

In [ ]:
# Q3 TODO: plot the distribution of the p values, ideally with ggplot



#### Check the ajusted p values
By default, p values are are adjusted by [Benjamini-Hochberg method, i.e., FDR](https://en.wikipedia.org/wiki/False_discovery_rate)

In [ ]:
ggplot(data = DEresults, aes(x = log10(padj/pvalue))) + 
  geom_histogram(bins = 30)

### 3.2 Compare to likelihood ratio test
In DESeq2, generalised linear model is used for DE gene detection, and there are two main tests to perform:
1. Wald test (default): estimating the mean and variance of the effect size, and then calculate the p value by Gaussian distribution. Null hypothesis: effect size is zero
2. Likelihood ratio test (taught in lecture): compare the likelihood ratio between two models: with vs without the candidate covariate, and then calculate the p value by Chi-square distribution. Null hypothesis: the likelihood between these two model are the same (or not very different).

In [ ]:
# fit the likelihood ratio test by specifying the null hypothesis with the reduced model
dds_LRT <- nbinomLRT(dds, reduced=as.formula('~ 1'))

In [ ]:
Wald_pval = rowData(dds)$WaldPvalue_group_CTRL_vs_CASE
LRT_pval = rowData(dds_LRT)$LRTPvalue

<font color='red'>**Q4: Plot the scatter plot of -log10 value between the Wald_pval and LRT_pval. What the difference did you see by comparing the p values between these two different tests? Write in report with attaching the figure. [3 marks]**</font>

Hints: consider sensitivity and potential false positives

In [ ]:
# Q4 TODO: Plot the scatter plot: -log10(Wald_pval) vs -log10(LRT_pval), ideally with ggplot



### 3.3 Multiple factors

The original column data

In [ ]:
col_data

#### Additional factor
As a hypothetical setting, we found out that the CASE_1, CASE_2 and CTRL_1, CTRL_2 are frozen samples for two weeks, and the rest are the fresh samples.

Now we want to consider the variations that comes from this additional factor, and how it affects the differential expression between normal and cancer

In [ ]:
col_data_2f <- col_data
col_data_2f$is_frozen <- c(1, 1, 0, 0, 0, 1, 1, 0, 0, 0)
col_data_2f

<font color='red'>**Q5.1: now write the script for DE analysis with additional covariate below using the defined designFormula [4 marks]**</font>

In [ ]:
#define the design formula
designFormula <- "~ group + is_frozen"

# Q5.1a TODO: create a DESeq dataset object from the count matrix and the updated col_data_2f and  designFormula
# ddsTwoFactor <- DESeqDataSetFromMatrix(PLEASE_FILL_HERE)


In [ ]:
# Q5.1b: TODO: Remove unexpressed genes
# ddsTwoFactor <- ddsTwoFactor[PLEASE_FILL_HERE]

#print dds object to see the contents
print(ddsTwoFactor)

In [ ]:
# Q5.1c TODO: run DESeq with the defined object
# ddsTwoFactor <- DESeq(PLEASE_FILL_HERE)


In [ ]:
# Q5.1d TODO: run DESeq with the defined object with likelhood ratio test
# hint: set parameter reduced=as.formula('~ is_frozen') in nbinomLRT function

# ddsTwoFactor_LRT <- nbinomLRT(PLEASE_FILL_HERE)


In [ ]:
# combine the pvalues into a data frame
df <- data.frame(One_factor_pval = rowData(dds_LRT)$LRTPvalue,
                 Two_factor_pval = rowData(ddsTwoFactor_LRT)$LRTPvalue)

<font color='red'>**Q5.2: Make a scatter plot between the -log10(One_factor_pval) and -log10(Two_factor_pval). What is the difference between p values when considering additional variable? What is the possible reason?  [2 marks]**</font>

Hints: consider the source of the variations

In [ ]:
# Q5.2 TODO: Plot the scatter plot: -log10(One_factor_pval), -log10(Two_factor_pval), ideally with ggplot



## 4. Gene set analysis

### Get DE genes

In [ ]:
#compute the contrast for the 'group' variable where 'CTRL' 
#samples are used as the control group. 

DE_res = results(dds_LRT, contrast = c("group", 'CASE', 'CTRL'))

#sort results by increasing p-value
DE_res <- as.data.frame(DE_res[order(DE_res$pvalue), ])

<font color='red'>**Q6.1: Write script to select DE genes by using padj < 0.05 and abs(DE_genes$log2FoldChange) > 1  [1 marks]**</font>

In [ ]:
# remove genes with NA values 
DE_genes <- DE_res[!is.na(DE_res$padj), ]

# Q6.1a TODO: select genes with adjusted p-values below 0.05
# DE_genes <- DE_genes[PLEASE_FILL_HERE]

# Q6.1b TODO: select genes with absolute log2 fold change above 1 (two-fold change)
# DE_genes <- DE_genes[PLEASE_FILL_HERE]

In [ ]:
head(DE_genes)
dim(DE_genes)

### GO enrichment analysis

In a typical differential expression analysis, thousands of genes are found differentially expressed between two groups of samples. Besides exploring individual gens, we can also calculate the overlap between DE genes and annotated gene sets for function association, e.g., Gene Ontology (GO) terms.

You could copy the above DE genes into the web server, e.g., [GO website](http://geneontology.org) or [David web server](https://david.ncifcrf.gov/).

Here, we show how to use R package `gProfileR` to perform this overlap enrichment analysis.

In [ ]:
# Only need install once
if (!requireNamespace("gProfileR", quietly = TRUE))
    install.packages("gProfileR")

In [ ]:
library(gProfileR)

In [ ]:
#get the list of genes of interest
genesOfInterest <- rownames(DE_genes)

#calculate enriched GO terms
goResults <- gprofiler(query = genesOfInterest, 
                       organism = 'hsapiens', 
                       src_filter = 'GO', 
                       hier_filtering = 'moderate')

<font color='red'>**Q6.2: Interpret the GO enriched terms and discuss its potential relevance to the cancer mechanism in the report  [2 marks]**</font>

In [ ]:
col_show <- c('p.value', 'term.size', 'query.size', 'overlap.size', 'term.name')
head(goResults[order(goResults$p.value), col_show], 10)

In [ ]:
sessionInfo()